<a href="https://colab.research.google.com/github/Laimo64/Laimo64/blob/main/AI_CW_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1y8OFnIFYC_b_WIexKSog732_L78Pegxa/view?usp=drive_link
!unzip -q brain_small.zip


Downloading...
From (original): https://drive.google.com/uc?id=1y8OFnIFYC_b_WIexKSog732_L78Pegxa
From (redirected): https://drive.google.com/uc?id=1y8OFnIFYC_b_WIexKSog732_L78Pegxa&confirm=t&uuid=3cd60f58-92d0-42e4-b826-fb255f4d2df9
To: /content/brain_small.zip
100% 119M/119M [00:03<00:00, 37.1MB/s]


In [3]:
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

class EarlyStopping:
    def __init__(self, patience=5, verbose=True, delta=0.00005, path="checkpoint.pt"):
        """
        Args:
            patience (int): 容忍驗證損失未改善的次數 (default: 5)
            verbose (bool): 是否打印相關資訊 (default: False)
            delta (float): 最小改善幅度，只有超過此值才算改善 (default: 0)
            path (str): 模型權重保存路徑 (default: "checkpoint.pt")
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float("inf")

    def __call__(self, val_loss, model):
        # 計算當前得分（驗證損失的負值，因為越小越好）
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """保存當前模型權重"""
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


def visualize_results(input_image, target_image, predicted_image, epoch, idx):
    """
    視覺化輸入影像、目標影像與預測影像
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # 輸入影像
    axes[0].imshow(input_image[0, 0, :, :, input_image.shape[4] // 2].cpu().detach().numpy(), cmap="gray")
    axes[0].set_title("Input (MR)")
    axes[0].axis("off")

    # 目標影像3
    axes[1].imshow(target_image[0, 0, :, :, target_image.shape[4] // 2].cpu().detach().numpy(), cmap="gray")
    axes[1].set_title("Target (CT)")
    axes[1].axis("off")

    # 預測影像
    axes[2].imshow(predicted_image[0, 0, :, :, predicted_image.shape[4] // 2].cpu().detach().numpy(), cmap="gray")
    axes[2].set_title("Prediction (Generated CT)")
    axes[2].axis("off")

    plt.suptitle(f"Epoch {epoch}, Batch {idx}")
    plt.show()

In [7]:
import os
import nibabel as nib
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Normalize
from scipy.ndimage import zoom


# Data preprocessing and dataset class
class MRCTDataset(Dataset):
    def __init__(self, data_dir, target_size=(128, 128, 128)):
        self.data_dir = data_dir
        self.target_size = target_size
        self.samples = [
            os.path.join(root)
            for root, _, files in os.walk(data_dir)
            if "mr.nii.gz" in files and "ct.nii.gz" in files
        ]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample_path = self.samples[idx]
        mr = nib.load(os.path.join(sample_path, "mr.nii.gz")).get_fdata()
        ct = nib.load(os.path.join(sample_path, "ct.nii.gz")).get_fdata()

        mr = self._normalize(mr)
        ct = self._normalize(ct)

        mr = self._resize_or_pad(mr, self.target_size)
        ct = self._resize_or_pad(ct, self.target_size)

        mr = torch.tensor(mr, dtype=torch.float32).unsqueeze(0)
        ct = torch.tensor(ct, dtype=torch.float32).unsqueeze(0)

        return mr, ct

    def _normalize(self, image):
        return (image - np.mean(image)) / (np.std(image) + 1e-5)

    def _resize_or_pad(self, image, target_size):
        zoom_factors = [t / s for t, s in zip(target_size, image.shape)]
        return zoom(image, zoom_factors, order=1)

# Generator and Discriminator Networks
class Generator(nn.Module):   # to transform MR to CT using 3D CNN
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose3d(64, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()   # 使範圍在[-1, 1]
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):  # to tell whether the input image is the ground truth from the dataset or generated by the generator
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv3d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm3d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv3d(128, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()       ###########查查為何跟generator用不同
        )

    def forward(self, x):
        return self.model(x)

# Training Loop
def train_gan(data_loader, generator, discriminator, g_optimizer, d_optimizer, criterion, epochs=10, device="cuda"):
    generator.to(device)
    discriminator.to(device)

    for epoch in range(epochs):
      for i, (mr, ct) in enumerate(data_loader):
          mr, ct = mr.to(device), ct.to(device)

          # Train Discriminator
          real_output = discriminator(ct)
          fake_ct = generator(mr)
          fake_output = discriminator(fake_ct.detach())

          d_loss_real = criterion(real_output, torch.ones_like(real_output))
          d_loss_fake = criterion(fake_output, torch.zeros_like(fake_output))
          d_loss = (d_loss_real + d_loss_fake) / 2

          d_optimizer.zero_grad()
          d_loss.backward()
          d_optimizer.step()

          # Train Generator
          fake_output = discriminator(fake_ct)
          g_loss = criterion(fake_output, torch.ones_like(fake_output))

          g_optimizer.zero_grad()
          g_loss.backward()
          g_optimizer.step()

          if i % 10 == 0:
              print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(data_loader)}], D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")
      visualize_results(mr, ct, fake_output, epoch, idx=1)

# Main Script
if __name__ == "__main__":
    data_dir = "/content/small"
    dataset = MRCTDataset(data_dir)
    data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

    generator = Generator()
    discriminator = Discriminator()

    criterion = nn.BCELoss()
    g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)

    train_gan(data_loader, generator, discriminator, g_optimizer, d_optimizer, criterion)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 116)